### import library

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import brotli
import pytz


### Get current time 

In [2]:
from datetime import datetime
import pytz

# Set the timezone to Taipei
taipei_tz = pytz.timezone('Asia/Taipei')

# Get current date and time in Taipei timezone
current_time_in_taipei = datetime.now(taipei_tz)

# convert to utc for unix time calculation
convert_utc = current_time_in_taipei.astimezone(pytz.utc)

# Calculate tomorrow's date
tomorrow = convert_utc + timedelta(days=1)

# Set the time to 05:59:59
tomorrow_5_59_59 = tomorrow.replace(hour=5, minute=59, second=59, microsecond=0)

# Convert to timestamp
tomorrow_5_59_59_timestamp = int(tomorrow_5_59_59.timestamp())

# Calculate past ten's date
past_ten = convert_utc - timedelta(days=10)

# Set the time to 06:00:00
past_ten_6_00_00 = past_ten.replace(hour=6, minute=0, second=0, microsecond=0)

# Convert to timestamp
past_ten_6_00_00_timestamp = int(past_ten_6_00_00.timestamp())

### Crawling Annue(鉅亨網)

In [3]:
import requests
import json

def fetch_news(page, start_at, end_at, limit):
    url = f"https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?page={page}&startAt={start_at}&endAt={end_at}&limit={limit}"
    headers = {
        "Accept": "application/json, text/plain, */*",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        # Add other headers if necessary
    }
    
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print('Invalid URL:', url)
        print('Status Code:', response.status_code)
        return None
    else:
        if response.headers.get('Content-Encoding') == 'br':
            decompressed_content = brotli.decompress(response.content).decode('utf-8')
        else:
            decompressed_content = response.text
        
        try:
            return json.loads(decompressed_content)
        except json.JSONDecodeError:
            print("Error decoding JSON")
            return None

a = input("Please enter 'ONLY ONE' stockID or 'ONLY ONE' stock name to retrieve related news:")


start_at = past_ten_6_00_00_timestamp
end_at = tomorrow_5_59_59_timestamp
limit = 30


page = 1
while True:
    news_data = fetch_news(page, start_at, end_at, limit)
    if not news_data or 'items' not in news_data or not news_data['items']['data']:
        # No more news data or invalid response
        break

    news = news_data['items']['data']
    for n in news:
        if a in n['stock'] or a in n['keyword']:
            title = n['title']
            url = "https://news.cnyes.com/news/id/" + str(n['newsId'])
            print(title, " ", url)
    
    page += 1  # Increment the page number for the next request

00919拉回可以加碼嗎?前10大成分透漏什麼端倪?   https://news.cnyes.com/news/id/5424666
【量大強漲股整理】台股震盪回檔，出現止穩訊號?元月仍有紅包行情可期?   https://news.cnyes.com/news/id/5424613
台股重挫300點破月線後第一季買還是賣? CES消費電子展指標股 台積電 聯發科 鴻海 大立光，元旦疫情升溫 防疫概念股 康那香 熱映 亞諾法   https://news.cnyes.com/news/id/5424587
【新台股龍捲風】金麗科過高！中國市場契機？！安國休息！換拉神盾！接下來看誰？   https://news.cnyes.com/news/id/5424574
〈台股盤後〉遇月線反壓量縮翻黑小跌9點 貨櫃三雄逆風揚帆   https://news.cnyes.com/news/id/5424478
選舉行情結束 ? 重點在明天 ! 電子休息傳產該看誰 ?   https://news.cnyes.com/news/id/5423745
大立光1/11召開法說 潛望鏡頭滲透率及毛利率發展受關注   https://news.cnyes.com/news/id/5423754
【量大強漲股整理】台股震盪回檔，有機會先蹲後跳，個股輪動快速下，該如何選股?   https://news.cnyes.com/news/id/5423748
〈台股盤後〉電子股權值大停電 大盤長黑摜破月線收跌294點   https://news.cnyes.com/news/id/5423650
〈台股盤中〉電子股殺盤重挫逾300點 海運、防疫股抗跌   https://news.cnyes.com/news/id/5423604
創高黑棒仍是多頭，台積電元月效應、鴻海補漲指標、聯發科 轉弱訊號、緯穎公開賣、奇鋐公開賺   https://news.cnyes.com/news/id/5423068
兩岸三地市值500強  台積電重登市值王寶座   https://news.cnyes.com/news/id/5422984
【量大強漲股整理】元月紅包行情可期?台股有機會先下後上? ETF夯，潛力股可鎖定誰?   https://news.cnyes.com/news/id/5423037